<a href="https://colab.research.google.com/github/julianamo93/CP1_2TDSZ-/blob/main/CP1_2TDSZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **CheckPoint 01 - 2TDSZ**

*   Dataset - HouseHold_Power
*   Descrição: Medição de consumo de energia elétrica em uma casa com taxa de amostragem de um minuto ao longo de um período de quase 04 anos. Diferentes grandezas elétricas e alguns valores de submedição estão
disponíveis.
*   Valores ausentes: Sim
*   Tamanho do dataset: 133MB
*   Objetivo: Regressão/Clustering

Importação de Bibliotecas para realização do CheckPoint 01. Bibliotecas padrão para análise e desenvolvimento de DL/ML.

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

## Importação do Dados

Importando os dados do dataset HouseHold Power.


In [2]:
df = pd.read_csv('/content/household_power_consumption.txt', sep=';')
df.head()

<ipython-input-2-8b38190bc575>:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/household_power_consumption.txt', sep=';')


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


## Análise Exploratória

Iniciando a análise exploratória (EDA), com os dados do dataset.

In [3]:
df.shape

(2075259, 9)

In [4]:
# df.describe() -- não utilizamos o describe() porque não temos dados numéricos,
# então vamos transformar os dados em numéricos para poder usar o describe()

In [4]:
100 * (25979 / 2075259)

1.2518437457686005

## Convertendo as Colunas para valores

In [5]:
# Converter as colunas 'object' para 'float64'
df = df.apply(pd.to_numeric, errors='coerce')
# df = df.replace('?', np.nan)
# df = df.astype('float64')

In [6]:
# verificando se existem células vazias e somando os registros encontrados...
df.isna().sum()

,0
Date,2075259
Time,2075259
Global_active_power,25979
Global_reactive_power,25979
Voltage,25979
Global_intensity,25979
Sub_metering_1,25979
Sub_metering_2,25979
Sub_metering_3,25979


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   float64
 1   Time                   float64
 2   Global_active_power    float64
 3   Global_reactive_power  float64
 4   Voltage                float64
 5   Global_intensity       float64
 6   Sub_metering_1         float64
 7   Sub_metering_2         float64
 8   Sub_metering_3         float64
dtypes: float64(9)
memory usage: 142.5 MB


In [8]:
# salvando um novo dataframe eliminando as células vazias
df1 =  df.dropna()

In [9]:
# Verificando se existem células vazias e somando os registros encontrados

df1.isna().sum()

,0
Date,0
Time,0
Global_active_power,0
Global_reactive_power,0
Voltage,0
Global_intensity,0
Sub_metering_1,0
Sub_metering_2,0
Sub_metering_3,0


In [10]:
df1.shape

(0, 9)

## Geração da Amostra de 5% dos Dados

*   Utilizando sample, que preserva nossas colunas e pega apenas a porcentagem escolhida.

In [11]:
# df2 ---> amostra de 5%
df2 = df1.sample(frac=0.05, random_state=10)

In [12]:
df2.shape

(0, 9)

## Aplicação do Principal Component Analysis (PCA) ---> Redução de Dimensionalidade

In [13]:
# Número original de atributos = 7
# Número de Componentes após a redução =
n = 5
pca = PCA(n_components=n)

In [17]:
X_pca = pca.fit_transform(df2)

ValueError: Found array with 0 sample(s) (shape=(0, 9)) while a minimum of 1 is required by PCA.

In [ ]:
X_pca.shape

In [ ]:
# O método "explained_variance_ratio_" ajuda a
# porcentagem de variância dos dados que consegue ser explicada por cada um dos componentes selecionados
pca.explained_variance_ratio_


In [ ]:
total_variancia = pca.explained_variance_ratio_.sum()
print(f'O modelo PCA com {n} variáveis explica{100*(total_variancia): .2f} % dos dados de entrada.')